# import lib

In [63]:
#Data management
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
import re
import string
import os
from tqdm import tqdm
import glob


#data visualize
import matplotlib.pyplot as plt
import seaborn as sns

pythai lib

In [64]:
!pip install pythainlp -qq
!pip install epitran -qq
!pip install sklearn_crfsuite -qq
!pip install tensorflow deepcut -qq
!pip install attacut -qq

     |████████████████████████████████| 148 kB 8.3 MB/s 
     |████████████████████████████████| 73 kB 1.8 MB/s 
     |████████████████████████████████| 1.2 MB 49.0 MB/s 
     |████████████████████████████████| 743 kB 7.0 MB/s 
     |████████████████████████████████| 2.0 MB 6.7 MB/s 
     |████████████████████████████████| 462 kB 49.6 MB/s 
     |████████████████████████████████| 1.3 MB 5.9 MB/s 
     |████████████████████████████████| 87 kB 6.7 MB/s 
     |████████████████████████████████| 473 kB 50.5 MB/s 
     |████████████████████████████████| 596 kB 36.6 MB/s 
     |████████████████████████████████| 45 kB 2.9 MB/s 


# Get LST20

In [ ]:
! wget https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz
! tar -xvzf /content/AIFORTHAI-LST20Corpus.tar.gz

In [66]:
def LST20_list(path = '/content/LST20_Corpus/test/*'):
  sentense_payload = []
  words_payload = []
  POS_payload = []
  NER_payload = []
  index = 0
  for i in glob.glob(path) :
    fp = open(i,"r")
    for line in fp.readlines() :
      if line == '\n' :
        continue
      else :
        data_ = line[:-2].split("\t")
        sentense_payload.append(index)
        words_payload.append(data_[0])
        POS_payload.append(data_[1])
        NER_payload.append(data_[2])
    index +=1
  return pd.DataFrame(
      {"sentence_id": sentense_payload, "words": words_payload, "POS": POS_payload,"NER":NER_payload}
  )

In [67]:
train_df = LST20_list(path = '/content/LST20_Corpus/train/*')
val_df = LST20_list(path = '/content/LST20_Corpus/eval/*')
test_df = LST20_list(path = '/content/LST20_Corpus/test/*')

In [68]:
uni = train_df["words"].unique()

In [69]:
from pythainlp import sent_tokenize, word_tokenize
text = "แปลกแต่จริงสิ่งนี้ไม่น่าจะเกิดขึ้นกับฉันบอกกับฉันว่าเธอทำได้ยังไงเปลี่ยนคนที่คอยปิดหัวใจให้รักเธอง่ายดายศีรษะ"
print("sent_tokenize:", sent_tokenize(text))
print("newmm    :", word_tokenize(text, engine="newmm"))
print("longest  :", word_tokenize(text, engine="longest"))
print("multi_cut:", word_tokenize(text, engine="multi_cut"))
print("deepcut  :", word_tokenize(text, engine="deepcut"))
print("atta     :", word_tokenize(text, engine="attacut"))
word_newmm =  word_tokenize(text, engine="newmm")
word_longest =  word_tokenize(text, engine="longest")
word_multi = word_tokenize(text, engine="multi_cut")
word_deepcut =  word_tokenize(text, engine="deepcut")
word_atta = word_tokenize(text, engine="attacut")

sent_tokenize: ['แปลกแต่จริงสิ่งนี้ไม่น่าจะเกิดขึ้นกับฉันบอกกับฉันว่าเธอทำได้ยังไงเปลี่ยนคนที่คอยปิดหัวใจให้รักเธอง่ายดายศีรษะ']
newmm    : ['แปลก', 'แต่', 'จริง', 'สิ่ง', 'นี้', 'ไม่', 'น่าจะ', 'เกิด', 'ขึ้นกับ', 'ฉัน', 'บอก', 'กับ', 'ฉัน', 'ว่า', 'เธอ', 'ทำได้', 'ยังไง', 'เปลี่ยนคน', 'ที่', 'คอย', 'ปิด', 'หัวใจ', 'ให้', 'รัก', 'เธอ', 'ง่ายดาย', 'ศีรษะ']
longest  : ['แปลก', 'แต่', 'จริง', 'สิ่ง', 'นี้', 'ไม่', 'น่าจะ', 'เกิดขึ้น', 'กับ', 'ฉัน', 'บอก', 'กับ', 'ฉัน', 'ว่า', 'เธอ', 'ทำได้', 'ยังไง', 'เปลี่ยนคน', 'ที่', 'คอย', 'ปิด', 'หัวใจ', 'ให้', 'รัก', 'เธอ', 'ง่ายดาย', 'ศีรษะ']
multi_cut: ['แปลก', 'แต่', 'จริง', 'สิ่ง', 'นี้', 'ไม่', 'น่าจะ', 'เกิดขึ้นกับ', 'ฉัน', 'บอกกับ', 'ฉัน', 'ว่า', 'เธอ', 'ทำได้', 'ยังไง', 'เปลี่ยนคนที่', 'คอย', 'ปิด', 'หัวใจ', 'ให้', 'รัก', 'เธอ', 'ง่ายดายศีรษะ']
deepcut  : ['แปลก', 'แต่', 'จริง', 'สิ่ง', 'นี้', 'ไม่', 'น่า', 'จะ', 'เกิด', 'ขึ้น', 'กับ', 'ฉัน', 'บอก', 'กับ', 'ฉัน', 'ว่า', 'เธอ', 'ทำ', 'ได้', 'ยัง', 'ไง', 'เปลี่ยน', 'คน', 'ที่', 'คอย', 'ปิด', '

In [ ]:
x = [True if c in uni or c==" " else False for c in word_atta]
x

# Corpus

pythai lib

In [72]:
from pythainlp.spell import NorvigSpellChecker
from pythainlp.corpus import download , get_corpus_path , get_corpus

SyntaxError: ignored

In [73]:
checker = NorvigSpellChecker()  # use default filter (remove any word with number or non-Thai character)
len(checker.dictionary())
checker.dictionary()

dict_items([('สัปต', 10), ('แสบ', 891), ('อรรธสระ', 4), ('ริต', 123), ('นับถือ', 3003), ('ผื่น', 359), ('เฮี่ยม', 2), ('สแวล', 7), ('งมงาย', 264), ('ไนท์', 146), ('หน้าซีด', 275), ('สตูดิโอ', 285), ('กล่าม', 2), ('ฃุด', 2), ('รสนิยม', 709), ('ลองดู', 426), ('เอฟเฟคท์', 11), ('อินฟราเรด', 126), ('ปตท', 3), ('โฐ', 2), ('ฟลิว', 5), ('ทั้งคู่', 2319), ('เทอร์เชียรี', 39), ('กรึ๊บ', 5), ('สัญญา', 11140), ('เซอร์โคเนียม', 3), ('พลังงาน', 5164), ('ป๊าบ', 60), ('ไปป์', 44), ('วอลตัน', 6), ('กตัญญู', 547), ('ล่น', 32), ('เต็มตา', 165), ('ธัมบ์', 5), ('ณุต', 2), ('แป๊ง', 3), ('โป', 926), ('ปอกลอก', 9), ('เข็มขัด', 411), ('กก', 1055), ('ม่านตา', 122), ('จ.ก.', 5), ('หยั่น', 4), ('ประสม', 232), ('กอสก์', 2), ('พู่กลิ่น', 3), ('หวู', 28), ('อุ่ง', 3), ('อห่ง', 2), ('วิวิต', 3), ('เบเรีย', 2), ('เส้นแรง', 7), ('อัปภาคย์', 2), ('บลาส', 8), ('ไหปลาร้า', 20), ('กวาร์', 12), ('ดราฟท์', 4), ('ยุทธปัจจัย', 8), ('เกิ่ง', 5), ('คึง', 17), ('แล่ะ', 4), ('วดล', 2), ('ซล็อต', 6), ('บอสช์', 2), ('ฉมาร์', 3), ('

In [74]:
from pythainlp.corpus import download , get_corpus_path , get_corpus

In [75]:
class corpus():
  def dow(self,name):
    download(name)
    path = get_corpus_path(name)
    cor = get_corpus(path)
    self.word = ["".join(x.split("\t")[:-1]) for x in cor]
    self.fre = ["".join(x.split("\t")[-1]) for x in cor]
    dict_cor = {"word":self.word,"fre":self.fre}
    self.df = pd.DataFrame(dict_cor,index=range(len(self.word))).set_index('word')
  
  def __init__(self,name):
    self.dow(name)

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,word):
    return self.df.loc[word]

In [100]:
checker = NorvigSpellChecker()  # use default filter (remove any word with number or non-Thai character)
unigrame = checker.dictionary()

In [ ]:
trigrame = corpus("tnc_trigram_word_freqs")
bigrame = corpus("tnc_bigram_word_freqs")

In [ ]:
trigrame.word

In [ ]:
text = "แปลกแต่จริงสิ่งนี้ไม่น่าจะเกิดขึ้นกับฉันบอกกับฉันว่าเธอทำได้ยังไงเปลี่ยนคนที่คอยปิดหัวใจให้รักเธอง่ายดายศีรษะ"
word_deepcut =  word_tokenize(text, engine="deepcut")

word_deepcut.insert(0,"<s/>")
word_deepcut.append("<s/>")

tri_words = trigrame.word
for tri_index in range(len(word_deepcut)-2):
  tri_word = "".join(word_deepcut[tri_index:tri_index+3])
  if tri_word not in tri_words:
    print(tri_word)
  else:
    print("pass : " + tri_word)

In [ ]:
def Correctto(triword):
  

In [107]:
checker.dictionary()

dict_items([('สัปต', 10), ('แสบ', 891), ('อรรธสระ', 4), ('ริต', 123), ('นับถือ', 3003), ('ผื่น', 359), ('เฮี่ยม', 2), ('สแวล', 7), ('งมงาย', 264), ('ไนท์', 146), ('หน้าซีด', 275), ('สตูดิโอ', 285), ('กล่าม', 2), ('ฃุด', 2), ('รสนิยม', 709), ('ลองดู', 426), ('เอฟเฟคท์', 11), ('อินฟราเรด', 126), ('ปตท', 3), ('โฐ', 2), ('ฟลิว', 5), ('ทั้งคู่', 2319), ('เทอร์เชียรี', 39), ('กรึ๊บ', 5), ('สัญญา', 11140), ('เซอร์โคเนียม', 3), ('พลังงาน', 5164), ('ป๊าบ', 60), ('ไปป์', 44), ('วอลตัน', 6), ('กตัญญู', 547), ('ล่น', 32), ('เต็มตา', 165), ('ธัมบ์', 5), ('ณุต', 2), ('แป๊ง', 3), ('โป', 926), ('ปอกลอก', 9), ('เข็มขัด', 411), ('กก', 1055), ('ม่านตา', 122), ('จ.ก.', 5), ('หยั่น', 4), ('ประสม', 232), ('กอสก์', 2), ('พู่กลิ่น', 3), ('หวู', 28), ('อุ่ง', 3), ('อห่ง', 2), ('วิวิต', 3), ('เบเรีย', 2), ('เส้นแรง', 7), ('อัปภาคย์', 2), ('บลาส', 8), ('ไหปลาร้า', 20), ('กวาร์', 12), ('ดราฟท์', 4), ('ยุทธปัจจัย', 8), ('เกิ่ง', 5), ('คึง', 17), ('แล่ะ', 4), ('วดล', 2), ('ซล็อต', 6), ('บอสช์', 2), ('ฉมาร์', 3), ('

In [97]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = ''
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [110]:
from pythainlp import thai_above_vowels, thai_tonemarks

In [112]:
thai_tonemarks

'่้๊๋'

In [111]:
thai_above_vowels

'ัิีึืํ็'

In [ ]:
edits1("nat")

In [108]:
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words) # if w in tri_words

In [109]:
known("จริงสิ่งนี้")

{'ง', 'จ', 'น', 'ร', 'ส', 'ิ', 'ี', '่', '้'}